# Malicious DNS-over-HTTP Traffic Detection using Autoencoders

Welcome to this Jupyter Notebook! We will explore the use of autoencoder for anomaly detection of malicious DoH traffic. The notebook will guide your through loading a DoH traffic dataset, designing an autoencoder, training an autoencoder,  and evaluating their effectiveness in identifying malicious traffic. 

## Load the dataset

We will be using the same training, testing, and evaluation dataset we used for our ML models. 


In [ ]:
# Import the Pandas library
import pandas as pd

# Load the datasets
data_training   = pd.read_csv('data_training.csv')
data_testing    = pd.read_csv('data_testing.csv')
data_evaluation = pd.read_csv('data_evaluation.csv')

In [ ]:
data_training

## Brief Overview of Autoencoders

Autoencoders are deep neural networks that can be used for denoising images, reducing the dimensionality of complex data, and anomaly detection. 

The main objective of the autoencoder is to recreate statistical features from observed normal DoH traffic with high accuracy on its output. If the input to the autoencoder is a normal DoH TCP connection, the reconstruction error will be small. Otherwise, the reconstruction error will be large. 

Autoencoders are formed by an encoder, a coder, and a decoder. 

![image.png](https://miro.medium.com/v2/resize:fit:720/format:webp/1*44eDEuZBEsmG_TCAKRI3Kw@2x.png)
Source: [Arden Dertat](https://towardsdatascience.com/applied-deep-learning-part-3-autoencoders-1c083af4d798)

The difference between the input and the output can be measured using the mean squared error

$$MSE = \frac{1}{n}(x_i-\hat{x}_i)^2$$

where $x_i$ is the $i$th element in the input sample, and $\hat{x}_i$ is the $i$ element in the output. If a sample has an $MSE$ larger than a threshold, we determine it is malicious. Otherwise, we determine it is normal. 


## The Autoencoder Class
The autoencoder is a more complex ML model. Due to the many possible variants, there is no library that provides a standard autoencoder. Instead, we need to build it from scratch. To this end, we use TensorFlow. TensorFlow (along with PyTorch) are the two most common libraries to implement neural networks. 



TensorFlow expects the class of our model to have tens of standards methods. Luckily, most of them are implemented in a base model class called Model. We only need to define the decoder, encoder, code, and how to glue them together. 

We define the ```Autoencoder``` class as an inherited class of the ```Model``` class. 

The ```__init__``` function initializes the autoencoder architecture. This is where we tell TensorFlow how many layers and how many neuros per layer we need. We also specify what type of non-linear functions we should use. 

The encoder is built with a for loop. The ```layer_output_sizes``` variables contains a list with the number of neurons per layers. The size of the list determines how many layers we have in the encoder. 

The code (or Embedding layer) is defined with the last element of the ```layer_output_sizes``` list. 

The decoder is defined as a mirror of the encoder by reversing the order of the elements in ```layer_output_sizes```.

The ```call``` function tells TensorFlow in what order the layers should be called. 

In [ ]:
# Import tensorflow modules
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Lambda, Dense, LeakyReLU, BatchNormalization, Input
from tensorflow.keras import Sequential, callbacks, losses
from tensorflow.nn import softmax
import copy


# Ignore errors about CUDA driver. We do not have GPUs :(

class Autoencoder(Model):
    def __init__(self, layer_output_sizes):
          super(Autoencoder, self).__init__()

          #self.layer_output_sizes = layer_output_sizes
          #workaround to above commented line that does not work for some reason
          #self.__dict__['layer_output_sizes'] = layer_output_sizes
          layer_output_sizes = copy.deepcopy(layer_output_sizes)

          # The Encoder
          self.encoder = None
          for level in layer_output_sizes[:-1]:
              if self.encoder is None:
                  self.encoder = Sequential(name='Encoder')
                  #self.encoder.add(InputLayer(input_shape=(int(level),)))
                  self.encoder.add(Dense(int(level), name='input_layer_encoder'))
              else:
                  self.encoder.add(Dense(int(level)))
                  self.encoder.add(BatchNormalization())
                  self.encoder.add(LeakyReLU())

          # The Embedding layer of the Encoder
          self.encoder.add(Dense(layer_output_sizes[-1]))

          # Reverse layer descriptions
          layer_output_sizes.reverse()

          # The Decoder
          self.decoder = None
          for level in layer_output_sizes[1:]:
              if self.decoder is None:
                  self.decoder = Sequential(name='Decoder')
                  #self.encoder.add(InputLayer(input_shape=(int(level),)))
                  self.decoder.add(Dense(int(level), name='input_layer_decoder'))
              else:
                  self.decoder.add(Dense(int(level)))
                  self.decoder.add(BatchNormalization())
                  self.decoder.add(LeakyReLU())

    def call(self, x):
      encoded = self.encoder(x)
      decoded = self.decoder(encoded)
      return decoded

    def model(self): #for visualization purposes only
        x = Input(shape=(16))
        return Model(inputs=[x], outputs=self.call(x))

## Initializing an Autoendoer object
The class is a template that we can "instantiate" everytime we need to create an autoencoder. For now, we only need one. Our autoencdoer will have one hidden layer with 14 neurons. The code layers will have 3 neurons. 

In [ ]:
model_architecture = [17,14,3] # the 16 corresponds to the number of features in our dataset. 
ae = Autoencoder(model_architecture)

## Training the autoencoder
Training deep neural networks involves two main concepts: loss function and training algorithm. 

The loss function is a function that measures how well the autoencoder is performing. In our case, we can use the mean squared error described above. This will tell us how well the autoencoder is recreating the DoH traffic data. 

The training algorithm is the set of steps that we take to update the parameters of the neural network based on how well it is doing. If the loss function is large, we make large changes. If the loss function, is small we make small changes. 

The most common training algorithm is ADAM optimization, which is based on Stochastic Gradient Descent. The good news is that you do not need to be familiar with this algorithm to train your autoencoder. However, you do need to be familiar with the concept of iterations. 

The ADAM algorithm updates the paramters of the autoencoder in iterations. Each iteration, the algorithm takes a set of training samples and asks the autoencoder to recreate them. It then updates the parameters according to the calculates the mean squared errors. It continues to pick random samples and updating the parameters until it reaches a maximum number of iterations. 

The maximum number of iterations is defined by how many times we want the algorithm to use the training data set. Every time the algorithm uses all the training data set is called an epoch. 


In [ ]:
# Set the training parameters
ae.compile(optimizer='adam', loss='mse')


# We create a callback to save the best performing autoencoder
# We use the mean squared error (i.e., loss function) to determine if when the model improves
# Smaller mean squared errors mean better models
cp_callback = callbacks.ModelCheckpoint(filepath='autoencoder_model/', save_weights_only=False, \
                                        monitor='mse', mode='min', save_best_only=True, verbose=1)

# Advnced visualization callback with tensorboard
# %load_ext tensorboard
# import tensorboard
# from tensorflow import keras
# tensorboard_callback = keras.callbacks.TensorBoard(log_dir='logs/')


# Uncomment the following line to use only a few training samples if your training is taking too long.  
data_training = data_training.sample(n=500)

# Training
# The fit function implements the ADAM algorithm for us
# We pass data_training twice. First is for the input. Second is to tell the ADAM algorithm how the output should
# look like. 
# validation_split tells the training algorithm what percentage of training data should be saved for validation. 
# epochs tells the ADAM algorithm when to stop
# batch_size tells the ADAM algorithm how manny samples to process per iteration
from time import perf_counter
tic = perf_counter()#start training timer
training_history =  ae.fit(data_training, data_training, validation_split=0.2, \
                           validation_data=None, epochs=3, batch_size=32, verbose=1, \
                           callbacks=[cp_callback,\
                                      #tensorboard_callback
                                     ])
toc = perf_counter()# stop training timer
training_time = toc-tic #calculate total training time

print("The training time of the Autoencoder was:")
training_time

## Deciding whether we need more or less training 
All machine learning models can suffer from underfitting and over fitting. Underfitting occurs when we do not learn enough from our trianing dataset due to a small number of epochs. Overtraining occurs when we train the model too much. This results in the model learning exactly our training dataset and loosing its ability to genralize its knowledge to useen samples. 

We can plot the mean squared error for training samples and for testing samples. We want the testing (aka validation) error to be minimum. 

In [ ]:
# Build a dataframe with the training history
history_dict = training_history.history
history_df= pd.DataFrame(data = history_dict)

# Plot the training history
fig = history_df.plot.line().get_figure()

# Advanced training history graphs
# %tensorboard --logdir logs


## Visuzalizing the autoencoder architecture
We initialized the autoencoder. But how does it actually look? We can visualize the layers using several methods. 
The easiest one is to use this [website](http://alexlenail.me/NN-SVG/index.html) to manually create the architecture. 

But that doesn't tell us exactly what we are training. 

Another option is to use the built-in tensorflow visualization library. This will show us what we are actually training. However, it is not very visually appealing. 

The most visually appealing and accurate option is Tensorboard. Unfortunately, it is also challenging to use. 

In [ ]:
tensorflow.keras.utils.plot_model(ae.model(), to_file='ae_arch.png', show_shapes=True, show_layer_names=True)

In [ ]:
# %tensorboard --logdir logs # Use at your own risk :)

### Updating the training parameters
Our model is underfitted. It can still learn more from the training dataset without hurting generalization.  We know this because both the training mean squared error (aka loss) and the testing mean squared error (aka validation loss) are both decreasing. 

Change the epochs and batch size parameters until you see your model overfitting. That is, when the validation loss starts to increase. 

### Compare the performance to the IF and LOF models. 